In [ ]:
# ==============================================================
# Task 2: End-to-End ML Pipeline with Scikit-learn Pipeline API
# Objective:
#   Build a reusable ML pipeline to predict customer churn
#   using the Telco Churn dataset.
# Skills:
#   - Preprocessing (scaling, encoding)
#   - Building Scikit-learn Pipelines
#   - Hyperparameter tuning (GridSearchCV)
#   - Export pipeline with joblib
# ==============================================================

# 1️⃣ Install Required Libraries (if not already installed)
# Uncomment and run these lines once if needed
# !pip install scikit-learn pandas joblib

# 2️⃣ Import Necessary Libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import joblib

# 3️⃣ Load Dataset (Telco Churn Dataset)
# 👉 Make sure you have 'Telco-Customer-Churn.csv' in your working directory
data = pd.read_csv("Telco-Customer-Churn.csv")

# 4️⃣ Basic Data Inspection
print("🔹 Dataset Shape:", data.shape)
print("🔹 Columns:", data.columns.tolist())
print(data.head())

# 5️⃣ Handle Missing Values (if any)
data = data.dropna()

# 6️⃣ Separate Features (X) and Target (y)
X = data.drop("Churn", axis=1)  # Features
y = data["Churn"]              # Target

# 7️⃣ Identify Categorical & Numerical Columns
categorical_cols = X.select_dtypes(include=["object"]).columns
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns

print("🔹 Categorical Columns:", categorical_cols.tolist())
print("🔹 Numerical Columns:", numerical_cols.tolist())

# 8️⃣ Preprocessing Pipelines
# - OneHotEncoder for categorical
# - StandardScaler for numerical
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
    ]
)

# 9️⃣ Create a Machine Learning Pipeline
# Here we try Logistic Regression and Random Forest
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000))
])

# 🔟 Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 1️⃣1️⃣ Define Models & Hyperparameters for GridSearchCV
param_grid = [
    {
        "classifier": [LogisticRegression(max_iter=1000)],
        "classifier__C": [0.1, 1.0, 10.0]
    },
    {
        "classifier": [RandomForestClassifier(random_state=42)],
        "classifier__n_estimators": [50, 100],
        "classifier__max_depth": [5, 10, None]
    }
]

# 1️⃣2️⃣ Apply GridSearchCV for Best Model Selection
grid_search = GridSearchCV(
    pipeline, param_grid, cv=3, n_jobs=-1, scoring="accuracy"
)

print("⏳ Training and tuning the models...")
grid_search.fit(X_train, y_train)

# 1️⃣3️⃣ Best Model from GridSearch
print("✅ Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# 1️⃣4️⃣ Model Evaluation
y_pred = best_model.predict(X_test)

print("\n🔹 Accuracy:", accuracy_score(y_test, y_pred))
print("🔹 F1 Score:", f1_score(y_test, y_pred, pos_label="Yes"))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# 1️⃣5️⃣ Export the Trained Pipeline
joblib.dump(best_model, "churn_pipeline.pkl")
print("📦 Model pipeline saved as churn_pipeline.pkl")
